In [1]:
import pandas as pd
import os
import re
import shutil
import tqdm

In [2]:
pattern = re.compile(r'([-\d]+)_c(\d+)')
train_path = '../RT-DETR/dataset/reid/bounding_box_train'
train_fns = os.listdir(train_path)
train_fns.sort()
seen_plushies = set()
query = []
gallery = []
for fn in train_fns:
    plushie_id = pattern.search(fn).groups()[0]
    if plushie_id not in seen_plushies:
        query.append(fn)
        seen_plushies.add(plushie_id)
    else:
        gallery.append(fn)
seen_plushies = [pattern.search(fn).groups()[0] for fn in query]  # re order to same as query
gallery_plushie_ids = [pattern.search(fn).groups()[0] for fn in gallery]  # re order to same as query

In [45]:
target_dir = '../RT-DETR/dataset/reid_train_testing'
os.makedirs(target_dir, exist_ok=True)
query_dir = os.path.join(target_dir, 'query')
gallery_dir = os.path.join(target_dir, 'bounding_box_test')
os.makedirs(query_dir, exist_ok=True)
os.makedirs(gallery_dir, exist_ok=True)
for f in tqdm.tqdm(gallery):
    shutil.copy(os.path.join(train_path, f), os.path.join(gallery_dir, f))
for f in tqdm.tqdm(query):
    shutil.copy(os.path.join(train_path, f), os.path.join(query_dir, f))

100%|██████████| 200/200 [00:00<00:00, 858.38it/s]


In [3]:
import pickle
with open('test_on_train_dist_mat.pkl', 'rb') as f:
    dist_mat = pickle.load(f)

In [9]:
import numpy as np
train_fns_np = np.array(train_fns)
gallery_plushie_ids_np = np.array(gallery_plushie_ids)
threshold = 0.75
dist_mat_bool = dist_mat <= threshold
class_acc = {}
overall_acc = 0
for i, plushie in enumerate(seen_plushies):
    preds = dist_mat_bool[i]
    truth = gallery_plushie_ids_np == seen_plushies[i]
    correct = preds & truth
    acc = correct.sum() / truth.sum()
    class_acc[plushie] = acc
    overall_acc += acc * truth.sum() / len(gallery_plushie_ids_np)
print(f'Overall acc: {overall_acc}')

Overall acc: 0.998267469680717


In [10]:
for c, acc in class_acc.items():
    if acc != 1:
        print(c, acc)

111 0.9830508474576272
125 0.9827586206896551
165 0.9622641509433962
35 0.9846153846153847
79 0.984375
7 0.75
